In [99]:
# -*- coding:utf-8 -*- 

dic = {('a',1):100, ('b',3):"yes", ('c',10):0.98} 
print "1",dic.get('d')
#print dic['d']  沒d,所以會出錯
dic[('d',55)] = 250 
print "2",dic 
dic.update({('e',22):123, ('f',6):789}) 
#dic {'a': 100, 'c': 0.98, 'b': 'yes', 'e': 123, 'd': 'new', 'f': 789}
print "3",sorted(dic,key = lambda x : x[1]) #只能針對key排序與顯示
for rec in sorted(dic,key = lambda x : x[1]): 
    print rec, dic[rec]

1 None
2 {('a', 1): 100, ('d', 55): 250, ('b', 3): 'yes', ('c', 10): 0.98}
3 [('a', 1), ('b', 3), ('f', 6), ('c', 10), ('e', 22), ('d', 55)]
('a', 1) 100
('b', 3) yes
('f', 6) 789
('c', 10) 0.98
('e', 22) 123
('d', 55) 250
<type 'list'>


In [2]:
import requests
from bs4 import BeautifulSoup
import re
rs = requests.session()  

cookie = 'ASP.NET_SessionId=h3ht1fnmnb3fdc55ka51x055; lawbank=555c9235c4a9440c9cb47ca3d714938c-2; _gat=1; _ga=GA1.3.816590347.1437135848; nrhbne=QC3AH1JaeS5ciqo0/9nftg==; JubFrm-pagebox=%5EcourtFullName%3DTPDV%60SLDV%60PCDV%60ILDV%60KLDV%60TYDV%60SCDV%60MLDV%60TCDV%60CHDV%60NTDV%60ULDV%60CYDV%60TNDV%60KSDV%60HLDV%60TTDV%60PTDV%60PHDV%60KMDV%60LCDV%60KSYV%60%5Ejud_case%3D%E5%A9%9A%5Ejcatagory%3D2%5Eissimple%3D-1%5Ejt%3D%E9%9B%A2%E5%A9%9A; x=i=LX6DBRIZbYke/hY2fzlawjwFLgn0mokfGqS/X2tmtwyFDw+4jydPQXdnVecyT3v9&j=ExKZIgGGiQD19KUxjSulmRd4cCS013mf16VUoBCxWainbmYufAN3bzZLoLqLFhwxOnTKxyGFWxF875EJIi2dLxJH6hbqBxjAtbFdhUFXhN0N52w88aupjAgf+1xanz35gqeqSqCyVcutjKPBYXx2S4NWdfslegbdREC9AfMjAm+zkZefg3LaDBQlB1tXv4gpeRNEF+2/vUrprNHpdwTTB7ldH0fldmjQzRJ40RhXbZas97JDbl8rytgzE7yXF3pU1aOS+CB+FuM2RsvvSTyLiNC49QoPH/FsVgTUtN2MDuzVeQ050f22OH0IQwjVQ0Gb'
header={'Cookie':cookie}

local_diction={
    '臺灣台北地方法院':1,
    '臺灣士林地方法院':2,
    '臺灣新北地方法院':3,
    '臺灣宜蘭地方法院':4,
    '臺灣基隆地方法院':5,
    '臺灣桃園地方法院':6,
    '臺灣新竹地方法院':7,
    '臺灣苗栗地方法院':8,
    '臺灣臺中地方法院':9,
    '臺灣彰化地方法院':10,
    '臺灣南投地方法院':11,
    '臺灣雲林地方法院':12,
    '臺灣嘉義地方法院':13,
    '臺灣臺南地方法院':14,
    '臺灣高雄地方法院':15,
    '臺灣花蓮地方法院':16,
    '臺灣臺東地方法院':17,
    '臺灣屏東地方法院':18,
    '臺灣澎湖地方法院':19,
    '臺灣金門地方法院':20,
    '臺灣連江地方法院':21,
    '臺灣高雄少年及家事法院地方法院':22
    }    

res = requests.get('http://fyjud.lawbank.com.tw/listcontent5.aspx', headers=header) #只抓的到第一頁
#res.encoding #查看網頁編碼
soup = BeautifulSoup(res.text)
linklist = []
filename = []
print soup.select('.page td')[0].text  #頁數
for ele in soup.select('.Table-List td a'):
    linklist.append(ele.attrs['href'])
    
for list_num in range(0,len(linklist)-1):
    rs_get =rs.get('http://fyjud.lawbank.com.tw/'+linklist[list_num],headers=header)
    response = rs_get.text.encode('utf8')
    response_soup = BeautifulSoup(response)
    '''
    test_str=response_soup.select('.Table-List td')[1].text
    print test_str
    print type(test_str)
    print dir(test_str)
    '''
    filenumber=response_soup.select('.Table-List td')[1].text
    fileid=filenumber.split(',')[0].encode('utf8')+'05'+filenumber.split(',')[2].encode('utf8')
    localation = soup.select('h4')[0].text #臺灣台北地方法院 裁判書 -- 民事類
    local_list = re.match(r'(.+?) ', localation) #臺灣台北地方法院
    local_str =','.join(local_list.groups(0))
    local_id=local_diction[local_str.encode('utf8')]
    
    filename.append('sample'+str(local_id)+'_'+fileid)
    judgedate=response_soup.select('.Table-List td')[3].text.encode('utf8') #【裁判日期】1040630
    judge_reason=response_soup.select('.Table-List td')[5].text.encode('utf8') #【裁判案由】離婚
    content=response_soup.select('.Table-List td')[8].text.encode('utf8')#【裁判全文】
    record=fileid+";\n\n"+judgedate+";\n\n"+judge_reason+";\n\n"+content
    f = open('E:\iii\DataSet\sample'+str(local_id)+'_'+\
             fileid+'.txt', 'w')
    f.writelines(record)
    f.close()

'''
104,家陸許,15
104,婚,108
104,家陸許,14
104,婚,216
104,婚,207
......
'''

共 3567 筆 第 1 / 179 頁


C:\Python27\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


'\n104,\xe5\xae\xb6\xe9\x99\xb8\xe8\xa8\xb1,15\n104,\xe5\xa9\x9a,108\n104,\xe5\xae\xb6\xe9\x99\xb8\xe8\xa8\xb1,14\n104,\xe5\xa9\x9a,216\n104,\xe5\xa9\x9a,207\n......\n'

In [3]:
'''
for ele in filename:
    print ele
print 2,filename[0]  #第一個filename檔案
'''
'''
import os
for dirPath, dirNames, fileNames in os.walk("E:\iii\DataSet\"):
    #print dirPath
    for f in fileNames: #fileNames是list
        #print f.encode('utf8') #f是str
        #print f.decode('utf8')
        #print ",".join(list(f))
        #for ele in list(f):
            #print ele
        #print fpath.encode('utf-8')
'''
print filename

['sample1_10405164', 'sample1_10405129', 'sample1_10405119', 'sample1_1040568', 'sample1_1040563', 'sample1_1040596', 'sample1_1040595', 'sample1_1040581', 'sample1_1040530', 'sample1_10305460', 'sample1_104053', 'sample1_1040565', 'sample1_10305453', 'sample1_10405168', 'sample1_10405162', 'sample1_1040582', 'sample1_1040571', 'sample1_1040570', 'sample1_1040552']


In [115]:
import re
a = soup.select('h4')[0].text #臺灣台北地方法院 裁判書 -- 民事類
m = re.match(r'(.+?) ', a) #臺灣台北地方法院
mm =','.join(m.groups(0))

In [51]:

import jieba
import jieba.posseg as pseg
import operator

#list直接印列表中文部分會印出unicode編碼，必須用迴圈印才能印出正常編碼，因為list類別python預設轉為unicode
#方便程式內溝通
filename_2=['sample1_10405164', 'sample1_10405129', 'sample1_10405119', 'sample1_1040568', 'sample1_1040563', 'sample1_1040596', 'sample1_1040595', 'sample1_1040581', 'sample1_1040530', 'sample1_10305460', 'sample1_104053', 'sample1_1040565', 'sample1_10305453', 'sample1_10405168', 'sample1_10405162', 'sample1_1040582', 'sample1_1040571', 'sample1_1040570', 'sample1_1040552']
fileword=""
dic = {}
result = []
jieba.set_dictionary("E:\iii\jieba\dict.txt.big_3.txt")

#for obj in filename:
    #str_convert = ''.join(obj.encode('utf-8'))
    #file_list = open('C:\\Users\\BigData\\Desktop\\DataSet\\'+ obj.decode('utf8'), 'r')
    #file_list_2= open('C:\\Users\\BigData\\Desktop\\DataSet\\'+ obj.decode('utf8')+'_2', 'w')
file_list = open('E:\\iii\\DataSet\\'+ filename_2[0]+'.txt', 'r')
file_list_2= open('E:\\iii\\DataSet\\'+ filename_2[0]+'.txt'+'_2', 'w')
#file_list = open('C:\\Users\\BigData\\Desktop\\DataSet\\sample103,婚,337.txt'.decode('utf8'), 'r')
#file_list_2= open('C:\\Users\\BigData\\Desktop\\DataSet\\sample103,婚,337_2.txt'.decode('utf8'), 'w')

for line in file_list.readlines():
    line_s=line.strip()
    result.append(line_s)
file_list_2.write(", ".join(result))
file_list_2.close()
file_list.close()
#result2 = "".join(result)
result2 = "10405164;1040803;離婚;臺灣臺北地方法院民事判決　　　　　　　104年度婚字第164號原　　　告　張震儀訴訟代理人　鍾宛蓉律師被　　　告　高麗上列當事人間請求離婚事件，本院於民國104年7月20日言詞辯論終結，判決如下：主  文准原告與被告離婚。訴訟費用由被告負擔。事實及理由一、被告經合法通知未於言詞辯論期日到場，核無民事訴訟法第386 條各款所列情形，按照家事事件法第51條準用民事訴訟法第385條第1項前段規定，爰依原告之聲請，由其一造辯論而為判決。二、原告起訴主張略以：兩造前於民國91年在美國公證結婚定居美國，復因被告父親生病，兩造即回國定居，就近照料。並於93年10月30日在國內舉行結婚儀式宴請賓客，嗣被告之父親於94年間病逝，被告執意爭取成為美國公民，而原告因在臺灣已有穩定工作，只好勉強同意被告單獨前往美國，兩造即自95年開始分居迄今長達八年，兩造分居期間，原告雙親及被告母親照顧責任及家庭經濟重擔皆由原告一人承擔，被告均仰賴原告及母親提供生活開銷。原告多次要求被告放棄爭取美國公民身分，結束長期分局情況，但被告不給予明確回答，無意願改善長期分居狀況，使原告日漸心灰意冷。100年6月間原告向被告提出結束婚姻之想法，被告亦於101年6月20日、7月10日、7月19日以電子郵件要求與原告辦理離婚，惟被告拒絕依照美國律師說明之程序辦理離婚，執意依照其所要求之繁瑣離婚程序，原告無法配合辦理，兩造因而難以協議辦理離婚。惟兩造持續分居各自生活，甚至逢年過節亦無互動，婚姻關係早已名存實亡。原告於103年3月24日起七度以電子郵件請求被告辦理離婚，惟被告均無回應，只得訴請法院判決結束徒具空明之婚姻，使兩造得以另覓幸福。兩造既已分居八年，多年無夫妻之實，亦無互動，形同陌路，婚姻之意義盡失；況兩造均無維持婚姻意願，應無復合之可能，顯有難以維持婚姻之重大事由存在，爰依民法第1052條第2項之規定請求離婚等語。並聲明：1、准原告與被告離婚。2、訴訟費用由被告負擔。三、被告未於言詞辯論期日到場，亦未提出書狀作何聲明或陳述。四、按有民法第 1052條第1項以外之重大事由，難以維持婚姻者，夫妻一方得請求離婚；但其事由應由夫妻一方負責者，僅他方得請求離婚，此觀之民法第 1052條第2項之規定自明。所謂有重大事由，難以維持婚姻者，係以婚姻是否已生破綻而無回復之希望為其判斷之標準，而婚姻是否已生破綻無回復之希望，則應依客觀之標準，即難以維持婚姻之事實，是否已達於倘處於同一境況，任何人均將喪失維持婚姻之意願而定，有最高法院95年度台上字第2924號判決意旨可資參照。又難以維持婚姻之重大事由應由夫妻之一方負責者，僅他方得請求離婚，則於難以維持婚姻之重大事由，夫妻雙方均須負責時，自應比較衡量雙方之有責程度後，僅責任較輕之一方，得向責任較重之他方請求離婚；如有責程度相同時，雙方即均得請求離婚，始符公平之旨。而婚姻係以夫妻之共同生活為目的，配偶應互信互賴、相互協力，以保持共同生活之圓滿、安全及幸福，因而夫妻應相互尊重以增進情感和諧及誠摯之相處，此為維持婚姻之基礎，若此基礎不復存在，致夫妻無法共同生活，無復合之可能者，即應認有難以維持婚姻之重大事由存在。經查，原告主張兩造於民國91年在美國公證結婚定居美國，復因被告父親生病，兩造返國定居，並於93年10月30日舉行結婚儀式及宴客，然因被告執意爭取成為美國公民，兩造因而自95年開始分居迄今長達八年等情，業據其提出兩造戶籍謄本、電子郵件為證，復據證人即原告之父張恕忠證述綦詳，並經本院調取入出境資訊連結作業查詢存卷可稽，堪信原告之主張為真實。本院審酌兩造自95年起即分居異國兩地，分居期間長達八年，雙方各自生活，逢年過節未有聯繫，近年僅因離婚事宜偶有聯繫，堪認兩造間就夫妻應相互協力保持夫妻共同生活之圓滿、安全及幸福之婚姻基礎已不存在，客觀上依兩造目前狀況，任何人均將喪失維持婚姻之意欲，足以構成難以維持婚姻之重大事由。而本件依上所述，認造成兩造婚姻破綻之發生及加深，兩造均有可歸責事由，且兩造之可歸責程度相當。是原告依民法第1052條第2項規定訴請判決離婚，依法即無不合，應予准許。五、據上論結，本件原告之訴為有理由，依家事事件法第51條、民事訴訟法第385條第1項、第78條，判決如主文。中    華    民    國   104    年    8     月    3     日家事法庭  法  官  薛嘉珩以上正本係照原本作成。如對本判決上訴，須於判決送達後20日內向本院提出上訴狀。如委任律師提起上訴者，應一併繳納上訴審裁判費。中    華    民    國   104    年    8     月    3     日書記官  張竣閔"
seg_list = jieba.cut_for_search(result2) 

print(",".join(seg_list))

seg_list = jieba.cut(result2) 
'''
words = pseg.cut(result2)
for word, flag in words:
    print('%s %s' % (word, flag))  #列印斷字與判斷出的詞性
'''
for ele in list(seg_list):
    if ele not in dic:
        dic[ele] = 1
    else:
        dic[ele] = dic[ele] + 1

sorted_x = sorted(dic.items(), key=operator.itemgetter(1), reverse=True) #type : list
print (','.join(str(v) for v in sorted_x)).decode('utf8')
print dir(','.join(str(v) for v in sorted_x))

'''
for ele in sorted_x[0:50]:
    print ele
'''

f = open('E:\\iii\\DataSet\\'+ filename_2[0]+'_TermFeq'+'.txt','w')
for key,value in sorted_x:
    con = key.encode('utf8')+','+str(value)+'\n'
    f.write(con)
f.close()


Building prefix dict from E:\iii\jieba\dict.txt.big_3.txt ...
DEBUG:jieba:Building prefix dict from E:\iii\jieba\dict.txt.big_3.txt ...
Loading model from cache c:\users\bigdata\appdata\local\temp\jieba.uf52a43fc8da656e96501a1c69ae51e91.cache
DEBUG:jieba:Loading model from cache c:\users\bigdata\appdata\local\temp\jieba.uf52a43fc8da656e96501a1c69ae51e91.cache
Loading model cost 0.000 seconds.
DEBUG:jieba:Loading model cost 0.000 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


10405164,;,1040803,;,離婚,;,臺,灣,臺,北地,方法,院民事,判決,　,　,　,　,　,　,　,104,年度,婚字,第,164,號原,　,　,　,告,　,張震,儀訴,訟代理,人,　,鍾,宛蓉律師,被,　,　,　,告,　,高麗,,上列,當事,人間,請求,離婚,事件,，,本院,於,民國,104,年,7,月,20,日言詞,辯論,終結,，,判決,如下,：,主, , ,文准,原告,與,被,告離婚,。,訴訟費,用,由,被,告,負擔,。,事實,及理,由,一,、,被告,經合法,通知,未,於,言,詞辯,論期,日,到場,，,核無民事,訴訟法,第,386, ,條各款,所列,情形,，,按,照家,事事件,法,第,51,條準,用民事,訴訟法,第,385,條第,1,項前,段,規定,，,爰,依,原告,之聲,請,，,由,其,一造,辯論,而,為,判決,。,二,、,原告,起訴,主張略,以,：,兩造,前,於,民國,91,年,在,美國,公證,結婚定,居美國,，,復,因,被,告父,親生病,，,兩造,即,回國定,居,，,就,近,照料,。,並,於,93,年,10,月,30,日,在,國內,舉行,結婚,儀式,宴,請,賓客,，,嗣,被,告,之,父親,於,94,年間,病逝,，,被告,執意爭,取成,為,美國,公民,，,而,原告,因,在,臺,灣已,有,穩定,工作,，,只,好,勉強,同意,被,告單,獨前,往美國,，,兩造,即,自,95,年,開始,分居,迄,今長,達八年,，,兩造,分居期,間,，,原告,雙親,及,被,告母,親照,顧責任,及家庭,經濟,重擔,皆,由,原告,一人承擔,，,被,告均,仰賴,原告,及,母親,提供,生活,開銷,。,原告,多次,要求,被,告放棄,爭取,美國,公民身,分,，,結束,長,期分局,情況,，,但,被,告不給,予明確,回答,，,無意願,改善長,期分,居狀況,，,使,原告,日漸,心灰意,冷,。,100,年,6,月間,原告,向,被,告,提出,結束,婚姻,之,想法,，,被告,亦,於,101,年,6,月,20,日,、,7,月,10,日,、,7,月,19,日以,電子,郵件,要求,與,原告,辦理,離婚,，,惟,被,告,拒絕,依照,美國律,師說,明之程序,辦理,離婚,，,執意,依照,其,所,要求,之,繁瑣,離婚,程序,，,原告,無法,配合,辦理,，,兩造,因,而,難以,協議,辦理,離婚,。,惟

In [23]:
import jieba.analyse
jieba.analyse.set_idf_path("E:\iii\jieba\idf.txt.big") # file_name
jieba.analyse.set_stop_words("E:\iii\jieba\stop_words.txt") # file_name
keyword = jieba.analyse.extract_tags(result2, topK=20, withWeight=False, allowPOS=())  #此為list
print ",".join(keyword)

維持,夫妻,美國,難以,分居,生活,定居,民法,104,95,1052,八年,言詞辯論,20,破綻,父親,幸福,家事,10,要求


In [14]:
import jieba
sentences = ["我喜欢吃土豆","土豆是个百搭的东西","我不喜欢今天雾霾的北京", 'costumer service']
# jieba.suggest_freq(‘百搭‘, True)
words = [list(jieba.cut(doc)) for doc in sentences]
#print(words)
sen = ""
for ele in words:
    for ele2 in ele:
        sen = sen + ele2 + ','
print sen
#print(', '.join(words))
print jieba.suggest_freq('雾霾', True)

我,喜欢,吃,土豆,土豆,是,个,百,搭,的,东西,我,不,喜欢,今天,雾,霾,的,北京,costumer, ,service,
1


In [43]:
import jieba.analyse
sentence="我喜欢吃土豆,土豆是个百搭的东西,我不喜欢今天雾霾的北京"
tup=jieba.analyse.extract_tags(sentence, topK=1, withWeight=False, allowPOS=())
s = "".join(tup)
print(s)

土豆


In [5]:
# -*- coding: utf-8 -*-
'''
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import os

#browser = webdriver.Firefox()
#browser = webdriver.Ie('IEDriverServer.exe')
browser = webdriver.Chrome(executable_path=r'C:\Python27\WebDriver\chromedriver.exe')
browser.get('http://www.google.com.tw/')
elem = browser.find_element_by_xpath("//input[@id='lst-ib']")
elem.send_keys(u'艾小克' + Keys.RETURN)
time.sleep(1)
links = browser.find_elements_by_xpath("//h3[@class='r']/a")
assert links[0].text == u'瓶水相逢- 艾小克', 'not match'   ##有錯
browser.close()'''

from selenium import webdriver

driver = webdriver.Chrome(executable_path=r'C:\Python27\WebDriver\chromedriver.exe')
#driver.get('http://fyjud.lawbank.com.tw/listcontent5.aspx', headers=header)
#browser.find_element_by_id("su").click()
driver.get('http://www.baidu.com/')
print driver.title  # 把页面title 打印出来
driver.quit()

百度一下，你就知道


In [25]:
pwd

u'C:\\Users\\BigData'

In [9]:
import xlwt
import xlrd

file = xlwt.Workbook('C:\Users\BigData\demo1.csv')
print dir(file)

table1 = file.add_sheet('sheet name')
#table1.write(0,0,'test')

data = xlrd.open_workbook('C:\Users\BigData\demo.xlsx')
data.sheet_names()
table = data.sheets()[0]


#nrows = table.nrows
#ncols = table.ncols
#print nrows,ncols

for i in range(0,30):
    col_A = 0
    col_B = 1
    col_C = 2
    row = i / 3
    value = table.cell(i,0).value
    if i % 3 == 0:
        table1.write(row,col_A,value)
        #table.put_cell(row, col_C, 1, value, 0)
    elif i % 3 == 1:
        table1.write(row,col_B,value)
        #table.put_cell(row, col_D, 1, value, 0)
    else:
        table1.write(row,col_C,value)
        #table.put_cell(row, col_E, 1, value, 0)

file.save('C:\Users\BigData\demo1.csv')


['_Workbook__active_sheet', '_Workbook__all_fonts_num_formats_xf_styles_rec', '_Workbook__all_links_rec', '_Workbook__backup_on_save', '_Workbook__backup_rec', '_Workbook__bof_rec', '_Workbook__bookbool_rec', '_Workbook__boundsheets_rec', '_Workbook__codepage_rec', '_Workbook__country_code', '_Workbook__country_rec', '_Workbook__custom_palette_b8', '_Workbook__datemode_rec', '_Workbook__dates_1904', '_Workbook__dsf_rec', '_Workbook__eof_rec', '_Workbook__ext_sst_rec', '_Workbook__first_tab_index', '_Workbook__fngroupcount_rec', '_Workbook__height_twips', '_Workbook__hide_obj_rec', '_Workbook__hpos_twips', '_Workbook__hscroll_visible', '_Workbook__intf_end_rec', '_Workbook__intf_hdr_rec', '_Workbook__intf_mms_rec', '_Workbook__obj_protect', '_Workbook__obj_protect_rec', '_Workbook__owner', '_Workbook__palette_rec', '_Workbook__password_rec', '_Workbook__precision_rec', '_Workbook__prot4rev_pass_rec', '_Workbook__prot4rev_rec', '_Workbook__protect', '_Workbook__protect_rec', '_Workbook__

"\ntable1 = file.add_sheet('sheet name')\n#table1.write(0,0,'test')\n\ndata = xlrd.open_workbook('C:\\Users\\BigData\\demo.xlsx')\ndata.sheet_names()\ntable = data.sheets()[0]\n\n\n#nrows = table.nrows\n#ncols = table.ncols\n#print nrows,ncols\n\nfor i in range(0,30):\n    col_A = 0\n    col_B = 1\n    col_C = 2\n    row = i / 3\n    value = table.cell(i,0).value\n    if i % 3 == 0:\n        table1.write(row,col_A,value)\n        #table.put_cell(row, col_C, 1, value, 0)\n    elif i % 3 == 1:\n        table1.write(row,col_B,value)\n        #table.put_cell(row, col_D, 1, value, 0)\n    else:\n        table1.write(row,col_C,value)\n        #table.put_cell(row, col_E, 1, value, 0)\n\nfile.save('C:\\Users\\BigData\\demo1.csv')\n"